In [2]:
import cv2
import math
import numpy as np

### 1. 확대 & 축소
이미지 피라미드(Image pyramid)  
가우시안 피라미드, 라플라시안 피라미드를 이용
- 가우시안 피라미드: 이미지를 생성하는 업샘플링과 다운 샘플링을 사용
- 라플라시안 피라미드: 가우시안 피라미드 레이어 간의 차이를 구하는 것


In [37]:
src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\ferris-wheel.jpg")
dst = src.copy()

for i in range(3):
    dst = cv2.pyrDown(dst)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 2. 이미지 크기 조절
- 이미지를 확대하는 경우에는 픽셀에 대한 보간법
- 이미지를 축소하는 경우에는 픽셀에 대한 병합법
- 이미지 확대 & 축소에서는 2배로 확대, 축소만 가능하지만 이미지 크기 조절함수는 사용자가 원하는 크기로 변환 가능

- 절대크기
- 상대크기

#### 이미지 크기 조절 함수의 보간법
1. 가장 가까운 이웃 보간법  
분수 픽셀 위치에서 가장 가까운 원본 픽셀을 결과 이미지의 픽셀값으로 사용.

2. **쌍 선형 보간법**  
분수 픽셀 위치에서 2x2 크기의 주변 원본 픽셀과 가까운 거리에 따라 선형적으로 가중치를 할당해서 결과 이미지의 픽셀값으로 사용.  
가장 많이 활용(확대)

+ 비트 단위 쌍 선형 보간법

3. **영역 보간법**
결과 이미지의 픽셀 위치를 입력 이미지의 픽셀 위치에 배치하고 겹치는 영역의 평균을 구해..  
축소하는 경우 주로 활용

4. **4x4 바이 큐빅 보간법**  
분수 픽셀 위치에서 4x4 크기의 주변 원본 픽셀을 3차 큐빅 스플라인으로 계산.  
확대에 주로 활용

5. 8x8 란초스 보간법  
8x8 크기의 주변 원본 픽셀을 계산해서 사용.


※ 분수 픽셀(fractional pixel)
정수 좌표가 아닌 실수 좌표에 있는 픽셀.

In [38]:
src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\car.png")
dst = src[280:310, 240:405]
dst = cv2.resize(dst, dsize=(256,256), interpolation=cv2.INTER_NEAREST) # 픽셀 깨지게 보임

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 3. 대칭 & 회전

- 2x3 회전 행렬 생성 함수  getRotationMatrix2D  
회전 축의 기준점 변경과 비율을 조정할 수 있음.


In [39]:
src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\glass.jpg")
height, width, _ = src.shape
center = (width/2, height/2)
angle = 90
scale = 0.5
matrix = cv2.getRotationMatrix2D(center, angle, scale)
matrix

array([[ 3.061617e-17,  5.000000e-01,  4.600000e+02],
       [-5.000000e-01,  3.061617e-17,  6.800000e+02]])

- 바운딩 공간은 이미지 좌측 상당(0,0)을 기준으로 새롭게 생성됨
- 이미지를 다시 중심으로 옮기기 위해 매핑 변환 행렬(matrix)에 변환의 차이를 추가로 포함
- 매핑 변환 행렬에 크기가 변형된 바운딩 공간의 중심점에서 기존 중심점 좌표를 감산한 값을 가산

In [40]:
radians = math.radians(angle)
sin = math.sin(radians)
cos = math.cos(radians)
bound_w = int((height * scale * abs(sin)) + (width * scale * abs(cos)))
bound_h = int((height * scale * abs(cos)) + (width * scale * abs(sin)))

matrix[0,2] += ((bound_w / 2) - center[0])
matrix[1,2] += ((bound_h / 2) - center[1])

dst = cv2.warpAffine(src, matrix, (bound_w, bound_h))

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 4. 기하학적 변환
이미지를 구성하는 픽셀 좌푯값의 위치를 재배치하는 과정

- 아핀 변환(Affine Transformation)
    - 2x3 행렬
    - 행렬 곱셈에 벡터 합을 활용
    - 변환 전의 서로 평행한 선은 변환 후에도 평행함을 유지
    - 세개의 좌표 활용
    
    - 아핀 맵 행렬 생성 함수 (GetAffineTransform)
    - 아핀 변환 함수 (warpAffine)
    
- 원근 변환(Perspective Transformation)
    - 3x3 행렬
    - 호모그래피(Homography)
    - 아핀 변환에서 유지되는 수평성은 유지 되지 않음.
    - 네개의 좌표 활용

   - 원근 맵 행렬 생성 함수(getPerspectiveTransform)
   - 원근 변환 함수 (warpPerspective)


※ 한 평면의 점을 다른 평면의 점으로 매핑해서 뒤틀림, 오목함등을 구현할 수 있는 변환

In [9]:
src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\clouds.jpg")
height, width, _ = src.shape

src_pts = np.array([[0, 0], [0, height], [width, height], [width, 0]], dtype=np.float32)    # 원근 맵 행렬 함수는 정밀도 CV_32F 값으로 사용하므로
dst_pts = np.array([[300, 300], [0, height - 200], [width - 100, height - 100], [900, 200]], dtype=np.float32)

M = cv2.getPerspectiveTransform(src_pts, dst_pts)
dst = cv2.warpPerspective(src, M, (width, height), borderValue=(255, 255, 255))

cv2.namedWindow("dst", flags=cv2.WINDOW_NORMAL)
cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows() 

### 5. 모폴로지 변환
- 영상 내 픽셀값 대체에 사용. 주로 노이즈 제거, 요소 결합 및 분리, 강도 피크 검출 등에 이용  
- 집합의 포함간계, 이동, 대칭, 여집합, 차집합 성질 이용
- 팽창(dilate)
    - 커널에 의해 커버된 영역 안의 모든 픽셀 값 중에서 최대값으로 대체
    - 어두운 영역이 줄어들고 밝은 영역이 늘어남
    - 스펙클이 거지며, 객체 내부의 홀이 사라짐
    - 노이즈 제거 후 줄어든 크기를 복구하고자 할때 주로 사용

- 침식(erode)
    - 커널에 의해 커버된 영역 안의 모든 픽셀 값 중에서 최솟값으로 대체
    - 밝은 영역이 줄어들고 어두운 영역이 늘어남
    - 스펙클이 사라지며, 객체 내부의 홀이 커짐.
    - 노이즈 제거에 주로 사용
- 모폴로지 연산은 커널의 영향을 크게 받음
- 커널의 형태에 따라 결과가 달라짐.
- 구조 요소 사용
    - 직사각형, 타원 십자 모형의 형태로도 활용
    - 커널의 형태, 사이즈, 고정점을  설정
    - 구조 요소 생성 함수(getStructureingElement)

In [11]:
src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\dandelion.jpg", cv2.IMREAD_GRAYSCALE)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5), anchor=(-1,-1))    # 커널 크기 5x5, 타원 형태, 고정점 중심
dst = cv2.erode(src, kernel, iterations=3)  # 모폴로지 침식 3회

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 6. 모폴로지 연산  
- 이진화된 이미지라면 팽창과 침식 연산으로도 우수한 결과를 얻지만 그레이스케일이나 다중 채널 이미지일 경우 더 복잡한 연산이 필요
    - 모폴로지 연산
- 모폴로지 연산 함수의 연산자 플래그
    - 팽창 연산
    - 침식 연산
    - 열림 연산  
    침식 연산 -> 팽창 연산  
    스펙클이 사라지면서 발생한 객체의 크기 감소를 원래대로 복구

    - 닫힘 연산  
    팽창 연산 -> 침식 연산  
    객체 내부의 홀이 사라지면서 발생한 크기 증가를 원래대로 복구
    
    - 모폴로지 그레이디언트  
    팽창 연산, 침식 연산 각각 적용 -> 감산  
    밝은 영역의 가장자리를 분리하며 그레이스케일 이미지가 가장 급격하게 변하는 곳에서 가장 높은 결과를 반환

    - 탑햇 연산  
    입력 이미지에 연림 연산을 적용한 이미지를 감산  
    열림 연산이 적용된 이미지는 스펙클이 사라지고 객체의 크기가 보존된 결과임  
    입력 이미지에서 감산하면 밝은 영역이 분리되어 사라졌던 스펙클이나 작은 부분들이 표시  
    -> 열림 연산에서 사라질 요소들을 표시
    
    - 블랙햇 연산  
    닫힘 연산을 적용한 이미지에 입력  이미지를 감산  
    닫힘 연산이 적용된 이미지는 객체 내부의 홀이 사라지고 객체의 크기가 보존된 결과임    
    이 결과에 입력 이미지를 감산하면 어두운 영역이 채워져 사라졌던 홀 등이 표시  
    -> 닫힘 연산에서 사라질 요소를 표시  
    
    - 히트미스 연산  
    단일 채널 이미지에서 활용(주로 이진화 이미지)  
    이미지의 전경이나 배경 픽셀의 특정 패턴을 찾는데 사용  
    구조 요소의 형태에 큰 영향을 받음.  
    커널 내부의 0은 해당 픽셀을 고려않음, 1은 해당 요소를 유지  
